Show the optimal solution and processing time for the following problem:

$\text{min} -4x-2y\\
x+y \le 8\\
8x+3y \ge -24\\
-6x+8y\le 48\\
3x+5y \le 15\\
x\le 3\\
y\ge 0
$




In [2]:
import time

## ortools

In [31]:
solvers = ['CLP','GLOP','Gurobi_LP']

In [32]:
from ortools.linear_solver import pywraplp

for solver_name in solvers:
    start  = time.time()
    solver = pywraplp.Solver.CreateSolver(solver_name)

    x = solver.NumVar(-10,10,'x')
    y = solver.NumVar(0,10,'y')

    solver.Add(   x +   y <=   8)
    solver.Add( 8*x + 3*y >= -24)
    solver.Add(-6*x + 8*y <=  48)
    solver.Add( 3*x + 5*y <=  15)
    solver.Add(   x       <=   3)
    solver.Add(         y >=   0)

    solver.Minimize(x+y)

    results = solver.Solve()

    x_val = x.solution_value()
    y_val = y.solution_value()

    end = time.time()

    print(f"Optimal value pair: ({x_val},{y_val})")
    print(f"Optimized result: {x_val+y_val}, obtained with {"ortools"} using {solver_name} in {end-start} seconds")

Optimal value pair: (-3.0,0.0)
Optimized result: -3.0, obtained with ortools using CLP in 0.0012288093566894531 seconds
Optimal value pair: (-3.000000000000001,0.0)
Optimized result: -3.000000000000001, obtained with ortools using GLOP in 0.0009667873382568359 seconds
Optimal value pair: (-3.0,0.0)
Optimized result: -3.0, obtained with ortools using Gurobi_LP in 0.012195348739624023 seconds


## Pyomo

In [48]:
solvers = ['glpk','gurobi']

In [55]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

for solver_name in solvers:
    start  = time.time()    
    model = pyo.ConcreteModel()

    model.x = pyo.Var(bounds=(-10,10))
    model.y = pyo.Var(bounds=(-10,10))

    x = model.x
    y = model.y

    model.C1 = pyo.Constraint(expr =    x +   y <=   8)
    model.C2 = pyo.Constraint(expr =  8*x + 3*y >= -24)
    model.C3 = pyo.Constraint(expr = -6*x + 8*y <=  48)
    model.C4 = pyo.Constraint(expr =  3*x + 5*y <=  15)
    model.C5 = pyo.Constraint(expr =    x       <=   3)
    model.C6 = pyo.Constraint(expr =          y >=   0)

    model.obj = pyo.Objective(expr = x   +   y, sense = minimize)

    opt = SolverFactory(solver_name)
    opt.solve(model)

    #model.pprint()

    x_val = pyo.value(x)
    y_val = pyo.value(y)

    end = time.time()

    print(f"Optimal value pair: ({x_val},{y_val})")
    print(f"Optimized result: {x_val+y_val}, obtained with {"pyomo"} using {solver_name} in {end-start} seconds")

Optimal value pair: (-3.0,0.0)
Optimized result: -3.0, obtained with pyomo using glpk in 1.1221954822540283 seconds
Optimal value pair: (-3.0,0.0)
Optimized result: -3.0, obtained with pyomo using gurobi in 0.11577677726745605 seconds


## Pulp

In [60]:
solvers = pl.listSolvers(onlyAvailable=True)
solvers

['GLPK_CMD', 'GUROBI', 'GUROBI_CMD', 'PULP_CBC_CMD']

In [64]:
import pulp as pl

for solver_name in solvers:
    start = time.time()
    model = pl.LpProblem('Ex',pl.LpMinimize)
    x = pl.LpVariable('x',-10,10)
    y = pl.LpVariable('y',-10,10)

    model +=    x +   y <=   8
    model +=  8*x + 3*y >= -24
    model += -6*x + 8*y <=  48
    model +=  3*x + 5*y <=  15
    model +=    x       <=   3
    model +=          y >=   0

    model += x+y
    
    solver = pl.getSolver(solver_name)
    status = model.solve(solver)

    x_val = pl.value(x)
    y_val = pl.value(y)
    end = time.time()

    print(f"Optimal value pair: ({x_val},{y_val})")
    print(f"Optimized result: {x_val+y_val}, obtained with {"pulp"} using {solver_name} in {end-start} seconds")

Optimal value pair: (-3.0,0.0)
Optimized result: -3.0, obtained with pulp using GLPK_CMD in 0.5298922061920166 seconds
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 2 columns and 10 nonzeros
Model fingerprint: 0xa3cbeb40
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [3e+00, 5e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.03s
Presolved: 4 rows, 2 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.0000000e+00   0.000000e+00   0.000000e+00      0s
       0   -3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds (0.00 work units)
Optimal objective -3.000000000e+00
Gur